In [40]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from cnn_finetune import make_model

from net import AGNet
from loss import AGLoss

from torch.autograd import Variable
from datagen import ListDataset

import os

In [41]:
batch_size = 32
test_batch_size = 64
epochs = 50
learning_rate = 0.01
momentum = 0.9
dropout = 0.3
start_epoch = 0
best_correct = 0
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

In [42]:
transform_train = transforms.Compose([
    transforms.CenterCrop(150),
    transforms.RandomCrop(150, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225)
    )
])
trainset = ListDataset(root='../data/UTKFace/', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=8)

transform_test = transforms.Compose([
    transforms.CenterCrop(150),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225))
])
testset = ListDataset(root='../data/test/', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=8)

In [43]:
net = AGNet()
net.cuda()
criterion = AGLoss()
# criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=1e-4)

In [44]:
# def accuracy(age_preds, age_targets, gender_preds, gender_targets):
def accuracy(gender_preds, gender_targets):

#     AGE_TOLERANCE = 3
#     age_prob = F.softmax(age_preds)
#     age_expect = torch.sum(Variable(torch.arange(1, 117)).cuda() * age_prob, 1)
    
#     age_correct = ((age_expect - age_targets.float()).abs() < AGE_TOLERANCE).int().sum().cpu().data[0]
    
    gender_preds = F.sigmoid(gender_preds)
    gender_preds = (gender_preds > 0.5).int()
    gender_correct = (gender_preds == gender_targets.int()).int().cpu().sum().data[0]
    return gender_correct
#     return age_correct, gender_correct

In [45]:
def train(epoch):
    net.train()
    train_loss = 0
    total = 0
    age_correct = 0
    gender_correct = 0
    for batch_idx, (inputs, age_targets, gender_targets) in enumerate(trainloader):
        inputs = Variable(inputs.cuda())
        age_targets = Variable(age_targets.cuda())
        gender_targets = Variable(gender_targets.cuda())
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # 
        age_preds, gender_preds = net(inputs)
        loss = criterion(gender_preds, gender_targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.data[0]
        
#         age_correct_i, gender_correct_i = accuracy(age_preds,
#                                                   age_targets, gender_preds,
#                                                   gender_targets)
        gender_correct_i = accuracy(gender_preds, gender_targets)
#         age_correct += age_correct_i
        gender_correct += gender_correct_i
        total += len(inputs)
        print('train_loss: %f | avg_loss: %f | gender_precise: %f (%d/%d) [%d/%d]'
             % (loss.data[0], train_loss/(batch_idx+1),
               100.*gender_correct/total, gender_correct, total,
               batch_idx+1, len(trainloader)))
#         print('train_loss: %.3f | avg_loss: %.3f | age_prec: %.3f (%d/%d) | gender_prec: %.3f (%d/%d)  [%d/%d]'  \
#             % (loss.data[0], train_loss/(batch_idx+1),      \
#                100.*age_correct/total, age_correct, total,  \
#                100.*gender_correct/total, gender_correct, total,    \
#                batch_idx+1, len(trainloader)))

In [46]:
# Test
def test(epoch):
    print('\nTest')
    net.eval()
    test_loss = 0
    total = 0
    age_correct = 0
    gender_correct = 0
    for batch_idx, (inputs, age_targets, gender_targets) in enumerate(testloader):
        inputs = Variable(inputs.cuda())
        age_targets = Variable(age_targets.cuda())
        gender_targets = Variable(gender_targets.cuda())

#         age_preds, gender_preds = net(inputs)
        age_preds, gender_preds = net(inputs)
#         loss = criterion(age_preds, age_targets, gender_preds, gender_targets)
#         loss = criterion(gender_preds, gender_targets)

#         test_loss += loss.data[0]
        gender_correct_i = accuracy(gender_preds, gender_targets)
#         age_correct_i, gender_correct_i = accuracy(
#             age_preds, age_targets, gender_preds, gender_targets)
#         age_correct += age_correct_i
        gender_correct += gender_correct_i
        total += len(inputs)
        print('gender_prec: %f (%d/%d) [%d/%d]' % (100.*gender_correct/total, gender_correct, total, batch_idx+1, len(testloader)))
#         print('test_loss: %f | avg_loss: %f | age_prec: %f (%d/%d) | gender_prec: %f (%d/%d)  [%d/%d]' \
#             % (loss.data[0], test_loss/(batch_idx+1),      \
#                100.*age_correct/total, age_correct, total,  \
#                100.*gender_correct/total, gender_correct, total, \
#                batch_idx+1, len(trainloader)))

    # Save checkpoint
    global best_correct
#     if age_correct + gender_correct > best_correct:
    if gender_correct > best_correct:
        print('Saving..')
        best_correct = gender_correct
        state = {
            'net': net.state_dict(),
            'correct': best_correct,
            'epoch': epoch,
        }
        if not os.path.isdir('../weights/checkpoint'):
            os.mkdir('../weights/checkpoint')
        torch.save(state, '..weights/checkpoint/ckpt.pth')

In [47]:
for epoch in range(start_epoch, start_epoch+10):
    print("Number epoch: {}".format(epoch))
    train(epoch)
    test(epoch)

Number epoch: 0


/home/neosai/Documents/projects/predict_age_gender/src/loss.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  print("gender_loss: {}".format(gender_loss.data[0]))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


gender_loss: 0.6942108273506165


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


train_loss: 0.694211 | avg_loss: 0.694211 | gender_precise: 39.000000 (50/128) [1/186]
gender_loss: 0.6932098269462585
train_loss: 0.693210 | avg_loss: 0.693710 | gender_precise: 44.000000 (113/256) [2/186]
gender_loss: 0.6929138898849487
train_loss: 0.692914 | avg_loss: 0.693445 | gender_precise: 47.000000 (181/384) [3/186]
gender_loss: 0.6931552290916443
train_loss: 0.693155 | avg_loss: 0.693372 | gender_precise: 47.000000 (245/512) [4/186]
gender_loss: 0.6931732296943665
train_loss: 0.693173 | avg_loss: 0.693333 | gender_precise: 48.000000 (308/640) [5/186]
gender_loss: 0.6933927536010742
train_loss: 0.693393 | avg_loss: 0.693343 | gender_precise: 47.000000 (364/768) [6/186]
gender_loss: 0.6932075023651123
train_loss: 0.693208 | avg_loss: 0.693323 | gender_precise: 47.000000 (424/896) [7/186]
gender_loss: 0.693169116973877
train_loss: 0.693169 | avg_loss: 0.693304 | gender_precise: 47.000000 (483/1024) [8/186]
gender_loss: 0.693134605884552
train_loss: 0.693135 | avg_loss: 0.693285 

...

train_loss: 0.166579 | avg_loss: 0.221046 | gender_precise: 90.000000 (11256/12416) [97/186]
gender_loss: 0.2835696339607239
train_loss: 0.283570 | avg_loss: 0.221684 | gender_precise: 90.000000 (11368/12544) [98/186]
gender_loss: 0.19875848293304443
train_loss: 0.198758 | avg_loss: 0.221452 | gender_precise: 90.000000 (11483/12672) [99/186]
gender_loss: 0.19460158050060272
train_loss: 0.194602 | avg_loss: 0.221184 | gender_precise: 90.000000 (11599/12800) [100/186]
gender_loss: 0.15435725450515747
train_loss: 0.154357 | avg_loss: 0.220522 | gender_precise: 90.000000 (11722/12928) [101/186]
gender_loss: 0.2818053364753723
train_loss: 0.281805 | avg_loss: 0.221123 | gender_precise: 90.000000 (11835/13056) [102/186]
gender_loss: 0.1925557404756546
train_loss: 0.192556 | avg_loss: 0.220846 | gender_precise: 90.000000 (11952/13184) [103/186]
gender_loss: 0.25864362716674805
train_loss: 0.258644 | avg_loss: 0.221209 | gender_precise: 90.000000 (12062/13312) [104/186]
gender_loss: 0.20496636

train_loss: 0.186569 | avg_loss: 0.220772 | gender_precise: 90.000000 (18765/20736) [162/186]
gender_loss: 0.1370554119348526
train_loss: 0.137055 | avg_loss: 0.220258 | gender_precise: 90.000000 (18884/20864) [163/186]
gender_loss: 0.3188832402229309
train_loss: 0.318883 | avg_loss: 0.220860 | gender_precise: 90.000000 (18994/20992) [164/186]
gender_loss: 0.21327313780784607
train_loss: 0.213273 | avg_loss: 0.220814 | gender_precise: 90.000000 (19110/21120) [165/186]
gender_loss: 0.23837226629257202
train_loss: 0.238372 | avg_loss: 0.220920 | gender_precise: 90.000000 (19223/21248) [166/186]
gender_loss: 0.30415794253349304
train_loss: 0.304158 | avg_loss: 0.221418 | gender_precise: 90.000000 (19333/21376) [167/186]
gender_loss: 0.2750428020954132
train_loss: 0.275043 | avg_loss: 0.221737 | gender_precise: 90.000000 (19441/21504) [168/186]
gender_loss: 0.19452346861362457
train_loss: 0.194523 | avg_loss: 0.221576 | gender_precise: 90.000000 (19557/21632) [169/186]
gender_loss: 0.21937

train_loss: 0.324362 | avg_loss: 0.201695 | gender_precise: 91.000000 (4907/5376) [42/186]
gender_loss: 0.2267947494983673
train_loss: 0.226795 | avg_loss: 0.202278 | gender_precise: 91.000000 (5024/5504) [43/186]
gender_loss: 0.18780189752578735
train_loss: 0.187802 | avg_loss: 0.201949 | gender_precise: 91.000000 (5143/5632) [44/186]
gender_loss: 0.19326958060264587
train_loss: 0.193270 | avg_loss: 0.201757 | gender_precise: 91.000000 (5258/5760) [45/186]
gender_loss: 0.17167869210243225
train_loss: 0.171679 | avg_loss: 0.201103 | gender_precise: 91.000000 (5376/5888) [46/186]
gender_loss: 0.17775772511959076
train_loss: 0.177758 | avg_loss: 0.200606 | gender_precise: 91.000000 (5494/6016) [47/186]
gender_loss: 0.295192688703537
train_loss: 0.295193 | avg_loss: 0.202577 | gender_precise: 91.000000 (5610/6144) [48/186]
gender_loss: 0.2308320701122284
train_loss: 0.230832 | avg_loss: 0.203153 | gender_precise: 91.000000 (5730/6272) [49/186]
gender_loss: 0.13377968966960907
train_loss: 

train_loss: 0.199778 | avg_loss: 0.210106 | gender_precise: 90.000000 (12574/13824) [108/186]
gender_loss: 0.20604848861694336
train_loss: 0.206048 | avg_loss: 0.210068 | gender_precise: 90.000000 (12692/13952) [109/186]
gender_loss: 0.20235905051231384
train_loss: 0.202359 | avg_loss: 0.209998 | gender_precise: 90.000000 (12812/14080) [110/186]
gender_loss: 0.17774313688278198
train_loss: 0.177743 | avg_loss: 0.209708 | gender_precise: 91.000000 (12933/14208) [111/186]
gender_loss: 0.25385263562202454
train_loss: 0.253853 | avg_loss: 0.210102 | gender_precise: 91.000000 (13047/14336) [112/186]
gender_loss: 0.27970972657203674
train_loss: 0.279710 | avg_loss: 0.210718 | gender_precise: 90.000000 (13156/14464) [113/186]
gender_loss: 0.21587315201759338
train_loss: 0.215873 | avg_loss: 0.210763 | gender_precise: 90.000000 (13269/14592) [114/186]
gender_loss: 0.19915196299552917
train_loss: 0.199152 | avg_loss: 0.210662 | gender_precise: 90.000000 (13385/14720) [115/186]
gender_loss: 0.17

train_loss: 0.193419 | avg_loss: 0.211423 | gender_precise: 90.000000 (20137/22144) [173/186]
gender_loss: 0.24388989806175232
train_loss: 0.243890 | avg_loss: 0.211610 | gender_precise: 90.000000 (20251/22272) [174/186]
gender_loss: 0.2728992998600006
train_loss: 0.272899 | avg_loss: 0.211960 | gender_precise: 90.000000 (20362/22400) [175/186]
gender_loss: 0.21734963357448578
train_loss: 0.217350 | avg_loss: 0.211990 | gender_precise: 90.000000 (20481/22528) [176/186]
gender_loss: 0.20179924368858337
train_loss: 0.201799 | avg_loss: 0.211933 | gender_precise: 90.000000 (20596/22656) [177/186]
gender_loss: 0.1646464616060257
train_loss: 0.164646 | avg_loss: 0.211667 | gender_precise: 90.000000 (20713/22784) [178/186]
gender_loss: 0.1549515575170517
train_loss: 0.154952 | avg_loss: 0.211350 | gender_precise: 90.000000 (20832/22912) [179/186]
gender_loss: 0.2360949069261551
train_loss: 0.236095 | avg_loss: 0.211488 | gender_precise: 90.000000 (20947/23040) [180/186]
gender_loss: 0.269897

## Test with a image

In [51]:
import cv2
from PIL import Image

In [50]:
img = cv2.imread("/home/neosai/Downloads/46402_1951-02-15_2009.jpg")

In [117]:
img = Image.open("/home/neosai/Downloads/download.jpeg")

In [118]:
input_img = transform_test(img)

In [120]:
input_img.shape

torch.Size([3, 150, 150])

In [121]:
input_img.unsqueeze_(0).shape

torch.Size([1, 3, 150, 150])

In [122]:
input_img.shape

torch.Size([1, 3, 150, 150])

In [123]:
a, b = net(input_img.cuda())

In [124]:
b

tensor([-3.8809], device='cuda:0', grad_fn=<ViewBackward>)

In [125]:
gender_preds = F.sigmoid(b)
gender_preds = (gender_preds > 0.5).int()

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [126]:
gender_preds

tensor([0], device='cuda:0', dtype=torch.int32)

In [1]:
import numpy as np

In [22]:
b = np.random.rand(128, 116)

In [23]:
b.shape

(128, 116)

In [8]:
a = np.sum(np.arange(1, 117) * b, 1)

In [9]:
a

array([3620.47253613])

In [27]:
[np.argmax(b[i]) for i in range(0, 116)]

[85,
 102,
 68,
 83,
 52,
 13,
 114,
 105,
 74,
 19,
 42,
 1,
 21,
 95,
 109,
 83,
 48,
 104,
 91,
 39,
 64,
 3,
 83,
 9,
 52,
 26,
 106,
 25,
 103,
 34,
 73,
 27,
 82,
 54,
 81,
 34,
 71,
 112,
 24,
 77,
 21,
 39,
 17,
 83,
 102,
 21,
 106,
 101,
 114,
 101,
 69,
 44,
 77,
 103,
 12,
 47,
 101,
 30,
 84,
 48,
 50,
 100,
 38,
 8,
 105,
 97,
 58,
 13,
 91,
 47,
 93,
 97,
 61,
 16,
 41,
 114,
 105,
 102,
 39,
 42,
 87,
 106,
 70,
 13,
 51,
 106,
 51,
 55,
 104,
 48,
 90,
 64,
 40,
 99,
 5,
 0,
 45,
 90,
 5,
 16,
 1,
 39,
 67,
 56,
 62,
 112,
 35,
 42,
 86,
 11,
 41,
 102,
 19,
 78,
 71,
 74]

In [25]:
b[0].argmax

<function ndarray.argmax>

In [26]:
np.argmax(b[0])

85

In [21]:
b[0][55]

0.7162959670689439

array([[0.48304033, 0.77516863, 0.97833413, 0.3871517 , 0.24357855,
        0.15773135, 0.53445515, 0.92273702, 0.13137669, 0.14347272,
        0.39719973, 0.95451412, 0.04828658, 0.97270312, 0.11151566,
        0.59834198, 0.20499689, 0.76103823, 0.11478081, 0.64958808,
        0.67512689, 0.85111405, 0.96028944, 0.50181462, 0.83416862,
        0.99322512, 0.60526135, 0.11691834, 0.27786215, 0.47205074,
        0.77459759, 0.06659621, 0.94212423, 0.27200115, 0.13643705,
        0.42923546, 0.59603223, 0.10297902, 0.3353779 , 0.73987667,
        0.70128138, 0.98503977, 0.99805381, 0.11922939, 0.72293112,
        0.24500441, 0.57040028, 0.03973017, 0.39401589, 0.05505866,
        0.44677099, 0.14949467, 0.79249604, 0.61423846, 0.8821324 ,
        0.99992684, 0.82861075, 0.99426677, 0.36245055, 0.19003802,
        0.91026761, 0.21190709, 0.21237699, 0.68614634, 0.65465492,
        0.9076976 , 0.13159407, 0.80854219, 0.24745792, 0.92374191,
        0.90822876, 0.06291042, 0.6143676 , 0.22

In [28]:
import torch

In [29]:

a = torch.rand(128, 116)

In [30]:
a

tensor([[0.7913, 0.4329, 0.3026,  ..., 0.6049, 0.1280, 0.6655],
        [0.0876, 0.1363, 0.9883,  ..., 0.1942, 0.0160, 0.9075],
        [0.3080, 0.1526, 0.0819,  ..., 0.0956, 0.0059, 0.6564],
        ...,
        [0.2536, 0.8113, 0.0131,  ..., 0.8667, 0.0907, 0.9448],
        [0.2560, 0.2959, 0.4659,  ..., 0.0078, 0.2909, 0.6288],
        [0.0643, 0.0982, 0.2114,  ..., 0.5803, 0.1505, 0.2233]])

In [31]:
b = [torch.argmax(a[i]) for i in range(0, 116)]

In [32]:
len(b)

116

In [33]:
type(b)

list

In [35]:
c = torch.from_numpy(np.array(b))

In [37]:
c.shape

torch.Size([116])